In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/movies_keywords.csv
/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/test.csv
/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/movies_metadata.csv
/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/train.csv
/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/sample_submission.csv


In [2]:
keywords=pd.read_csv('/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/movies_keywords.csv')
meta_data=pd.read_csv('/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/movies_metadata.csv')
train=pd.read_csv('/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/train.csv')
test=pd.read_csv('/Users/yw511/VSCode/Machine Learning/kaggle-movie-recommendation/data/test.csv')

In [3]:
import sklearn

In [4]:
train[['userId','id']] = train.userId_movieId.str.split("_", expand=True)

train = train.drop(['userId_movieId'], axis=1)

train['rating'] = train['rating'].astype(float)
train['userId'] = train['userId'].astype(int)
train['id'] = train['id'].astype(int)

test[['userId','id']] = test.userId_movieId.str.split("_", expand=True)
test = test.drop(['userId_movieId'], axis=1)

train = train[['userId', 'id', 'rating']]
test = test[['userId', 'id']]

# User Based

# Citation

https://towardsdatascience.com/the-4-recommendation-engines-that-can-predict-your-movie-tastes-109dc4e10c52

In [5]:
from sklearn.model_selection import train_test_split as cv
train_data, test_data = cv(train, test_size=0.3)

In [6]:
# Create two user-item matrices for training and testing data
train_data_matrix = train_data.to_numpy()
test_data_matrix = test_data.to_numpy()

In [7]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data_matrix, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
# Item Similarity Matrix
# item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
# item_correlation[np.isnan(item_correlation)] = 0

In [8]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [10]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')

# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))


User-based CF RMSE: 12186.168642238565


In [12]:
user_prediction

array([[-3303.83683253, 10084.13278825, -3624.19595572],
       [ 4533.05045934, -8794.9333148 ,  4606.68285546],
       [ 4562.43038982, -9209.70862812,  4679.0782383 ],
       ...,
       [13366.49540496, 26904.65806549, 13050.74652955],
       [-3372.23405563,  8921.27527162, -3712.34121599],
       [-3865.7054972 ,  8879.41411361, -4199.90861641]])